In [1]:
!pip install pandas openpyxl xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 3.4 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded1 = files.upload()
uploaded2 = files.upload()

Saving SICU and MICU Triage Notes.xlsx to SICU and MICU Triage Notes.xlsx


Saving Outcome of Patient.xlsx to Outcome of Patient.xlsx


In [3]:
import pandas as pd

In [65]:
triage  = pd.read_excel('SICU and MICU Triage Notes.xlsx')
outcome = pd.read_excel('Outcome of Patient.xlsx')


In [66]:
raw = pd.read_excel('SICU and MICU Triage Notes.xlsx')

# 2) Parse the two pieces and combine
raw['Authored Date'] = pd.to_datetime(raw['Authored Date'], dayfirst=False)
raw['Authored Time'] = raw['Authored Time'].astype(str)  # ensure string like '14:52:00'
raw['Authored DateTime'] = pd.to_datetime(
    raw['Authored Date'].dt.date.astype(str) + ' ' + raw['Authored Time']
)

# 3) Filter by FY2019–20
mask = (
    (raw['Authored DateTime'] >= '2019-04-01') &
    (raw['Authored DateTime'] <= '2021-03-31')
)
triage = raw.loc[mask].copy()

# 4) Verify you now have rows
print("Rows after FY filter:", triage.shape[0])
print(triage[['Authored DateTime','Document Name']].head())

print("Total rows in master:", master.shape[0])

Rows after FY filter: 3318
    Authored DateTime         Document Name
0 2020-10-07 10:06:00  MICU Triage Note SGH
1 2020-12-28 17:02:00  MICU Triage Note SGH
2 2020-09-30 09:07:00  MICU Triage Note SGH
3 2021-03-11 03:50:00  MICU Triage Note SGH
4 2021-01-26 14:52:00  MICU Triage Note SGH
Total rows in master: 3377


In [67]:
# 5) Map Document Name → Team
triage['Team'] = pd.NA
triage.loc[triage['Document Name'].str.contains('SICU', na=False), 'Team'] = 'HERO Team'
triage.loc[triage['Document Name'].str.contains('MICU', na=False), 'Team'] = 'SMART Team'

print("Total rows in master:", master.shape[0])

Total rows in master: 3377


In [68]:
ph_dates = [
    # FY2019
    '2019-04-19','2019-05-01','2019-05-19','2019-05-20',
    '2019-06-05','2019-08-09','2019-08-11','2019-08-12',
    '2019-10-27','2019-10-28','2019-12-25','2020-01-01',
    '2020-01-25','2020-01-26','2020-01-27',
    # FY2020
    '2020-04-10','2020-05-01','2020-05-07','2020-05-24',
    '2020-05-25','2020-07-31','2020-08-09','2020-08-10',
    '2020-11-14','2020-12-25','2021-01-01','2021-02-12',
    '2021-02-13'
]
public_holidays = set(pd.to_datetime(ph_dates).date)

# 6) Classify Office vs After Hours (using Authored DateTime)
triage['Hour']    = triage['Authored DateTime'].dt.hour
triage['Weekday'] = triage['Authored DateTime'].dt.weekday  # Monday=0, Sunday=6
triage['Date']    = triage['Authored DateTime'].dt.date

def time_cat(row):
    d = row['Weekday']
    h = row['Hour']
    dt = row['Date']
    # 1) If Sunday or public holiday → After Hours
    if d == 6 or dt in public_holidays:
        return 'After Hours'
    # 2) Otherwise if Mon–Fri 08–17 or Sat 08–12 → Office Hours
    if (d < 5 and 8 <= h < 17) or (d == 5 and 8 <= h < 12):
        return 'Office Hours'
    # 3) Else → After Hours
    return 'After Hours'

triage['Time Category'] = triage.apply(time_cat, axis=1)

# Quick check
print(triage['Time Category'].value_counts())

print("Total rows in master:", master.shape[0])



Time Category
After Hours     1891
Office Hours    1427
Name: count, dtype: int64
Total rows in master: 3377


In [69]:
# 7) Merge with outcome
outcome = pd.read_excel('Outcome of Patient.xlsx')
master = triage.merge(outcome, on='Case No', how='left')
print("Rows after FY filter:", triage.shape[0])

Rows after FY filter: 3318


In [70]:

# 8) Create FY Month
master['FY Month'] = master['Authored DateTime'].dt.to_period('M').astype(str)
print("Rows after FY filter:", triage.shape[0])

print("Total rows in master:", master.shape[0])

Rows after FY filter: 3318
Total rows in master: 3377


In [71]:
master['SGH_Disposition'] = (
    master['SGH_Disposition']
      .astype(str)
      .str.strip()
)

# 2) Define your mapping exactly as per the spec
disp_map = {
    # General Ward
    'GW'       : 'GW',

    # Intermediate Care Area
    'MICA'     : 'ICA',
    'RICA'     : 'ICA',
    'NEM ICA'  : 'ICA',

    # High‑Dependency Unit
    'Burns HD' : 'HDU',
    'Iso HD'   : 'HDU',

    # Intensive Care Unit
    'Iso ICU'  : 'ICU',
    'MICU'     : 'ICU',
    'SICU'     : 'ICU',
    'NESICU'   : 'ICU',
    'BICU'     : 'ICU',
    'CCU'      : 'ICU',
    'CTSICU'   : 'ICU',
}

# 3) Map and fill any unmapped dispositions as “Other” (or NaN, if you prefer)
master['Disposition Group'] = (
    master['SGH_Disposition']
      .map(disp_map)
      .fillna('Other')
)

# 4) Verify the result
print(master['Disposition Group'].value_counts(dropna=False))

print("Total rows in master:", master.shape[0])

Disposition Group
GW       1090
ICU       827
Other     652
ICA       631
HDU       177
Name: count, dtype: int64
Total rows in master: 3377


In [72]:
# 1) Clean up the raw discharge descriptions
master['Discharge Type Description'] = (
    master['Discharge Type Description']
      .astype(str)
      .str.strip()
      # normalize hyphens just in case
      .str.replace('–', '-', regex=False)
      .str.replace('—', '-', regex=False)
)

# 2) Define the mapping exactly as per your table
discharge_map = {
    # Death
    'Death'                          : 'Death',
    'Death - Coroner Case'          : 'Death',

    # Follow‑up at Clinic
    'Follow-up at SOC'               : 'Follow-up at Clinic',
    'Follow-up at PHC Clinic'       : 'Follow-up at Clinic',
    'Discharge to Private GP/Specialist': 'Follow-up at Clinic',

    # Transferred to Other Hospitals
    'Transfer or Discharge to Centres_DisTyp': 'Transferred to Other Hospitals',
    'Transfer to Restructured Hospital'      : 'Transferred to Other Hospitals',
    'Transfer to Private Hospital'           : 'Transferred to Other Hospitals',

    # AOR (Against Medical Advice / Other Refusal)
    'Discharge against advice'      : 'AOR',
    'AOR - NO SOC F/P'              : 'AOR',

    # Residential Stepdown
    'Residential Stepdown Facilities': 'Residential Stepdown',

    # Others
    'Others'                        : 'Others',

    # Absconded
    'Absconded'                     : 'Absconded',

    # Discharged (patient sent home)
    'Patient Discharged'            : 'Discharged'
}

# 3) Map and fill any unmapped as “Others”
master['Discharge Group'] = (
    master['Discharge Type Description']
      .map(discharge_map)
      .fillna('Others')
)

# 4) Verify no blanks remain
print(master['Discharge Group'].value_counts(dropna=False))

print("Total rows in master:", master.shape[0])

Discharge Group
Follow-up at Clinic               1640
Death                             1041
Residential Stepdown               365
Transferred to Other Hospitals      98
AOR                                 87
Discharged                          79
Others                              61
Absconded                            6
Name: count, dtype: int64
Total rows in master: 3377


In [73]:
raw = raw.dropna(subset=['Case No'], how='all')

print("Total rows in master:", master.shape[0])

outcome = outcome.drop_duplicates(subset=['Case No'])
with pd.ExcelWriter('RRT_Masterfile.xlsx', engine='xlsxwriter') as writer:
    master.to_excel(writer,
                    sheet_name='Masterfile',
                    index=False)
print(master.head())
print("✅ Final Excel created: RRT_Masterfile.xlsx")

Total rows in master: 3377
  Case No         Document Name Authored Date Authored Time SGH_Disposition  \
0   5690G  MICU Triage Note SGH    2020-10-07      10:06:00         Iso ICU   
1   4269F  MICU Triage Note SGH    2020-12-28      17:02:00              GW   
2   1225E  MICU Triage Note SGH    2020-09-30      09:07:00              GW   
3   6166C  MICU Triage Note SGH    2021-03-11      03:50:00         Iso ICU   
4   2285D  MICU Triage Note SGH    2021-01-26      14:52:00            MICA   

  SGH_Time_Referred   Authored DateTime        Team  Hour  Weekday  \
0             08:38 2020-10-07 10:06:00  SMART Team    10        2   
1             16:04 2020-12-28 17:02:00  SMART Team    17        0   
2             08:48 2020-09-30 09:07:00  SMART Team     9        2   
3             02:55 2021-03-11 03:50:00  SMART Team     3        3   
4             14:52 2021-01-26 14:52:00  SMART Team    14        1   

         Date Time Category Discharge Type Description FY Month  \
0  2020-10